# OpenAI API 7 — 멀티소스 RAG (YouTube · Web · News)


## 학습 목표
- 외부 데이터(YouTube, 웹, 뉴스)를 활용한 확장형 RAG 구조 이해
- LangChain과 OpenAI API를 통한 유튜브 자막 요약
- 실시간 웹 검색 + 뉴스 요약 결합 실습
- 멀티소스 검색 기반 Q&A/리포트 파이프라인 설계


## 0. 환경 준비 (필요시 실행)

In [ ]:

# 필요시 다음을 실행하세요 (노트북/터미널 둘 다 가능)
# %pip install --upgrade langchain langchain-community langchain-openai openai tiktoken #     youtube-transcript-api feedparser tavily-python
# API Key 준비:
# - OpenAI:  export OPENAI_API_KEY=sk-...   (Windows: set OPENAI_API_KEY=...)
# - Tavily:  export TAVILY_API_KEY=tvly-...
print("필요 패키지: langchain, langchain-community, langchain-openai, openai, youtube-transcript-api, feedparser, tavily-python")

## 1. 공통 설정 및 안전한 임포트

In [ ]:
import os

try:
    from langchain_openai import ChatOpenAI
except Exception:
    from langchain.chat_models import ChatOpenAI  # type: ignore

try:
    from langchain_community.tools.tavily_search import TavilySearchResults
except Exception:
    from langchain.tools import TavilySearchResults  # type: ignore

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
print("Imports OK")

## 2. 유튜브 영상 요약


| 항목 | 설명 |
|------|------|
| 목적 | 유튜브 자막을 불러와 핵심 내용 요약 |
| 주요 라이브러리 | `youtube_transcript_api`, `LangChain`, `OpenAI` |
| 응용 예시 | 강의 요약, 세미나 요약, 뉴스 인터뷰 자동 요약 |


In [ ]:
# !pip install youtube_transcript_api

In [ ]:
#  라이브러리 로드
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableSequence
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA

#  환경 변수 로드
# load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


#  텍스트 분할 함수
def split_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(text)

#  요약 프롬프트
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
다음 텍스트 내용을 한국어로 요약해 주세요:

{text}
""")

#  LLM 및 요약 체인 정의
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
summary_chain = summary_prompt | llm | StrOutputParser()

### 유튜브 내용 요약하기
video_id = "4pYzYmSdSH4"  # 유튜브 링크의 v= 부분


In [ ]:
## 유튜브 자막 추출하기
print("▶ 유튜브 자막 추출 중... ", end='')

ytt_api = YouTubeTranscriptApi()
transcripts = ytt_api.fetch(video_id, languages=['ko', 'en'])
transcripts

In [ ]:
transcripts.snippets

In [ ]:
transcript = " ".join([entry.text for entry in transcripts.snippets])
print("@ 유튜브 자막 추출 완료...!!! ")
print("\n▶ 자막 추출 결과 (1000글자):\n", transcript[:1000])


In [ ]:
# pip install youtube-transcript-api --upgrade

from youtube_transcript_api import YouTubeTranscriptApi
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

video_id = "dQw4w9WgXcQ"  # 원하는 영상 ID

transcript = " ".join([entry.text for entry in transcripts.snippets])
print("@ 유튜브 자막 추출 완료...!!! ")

# LLM 기반 요약
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
prompt = PromptTemplate(
    template="다음 유튜브 자막을 5문장으로 요약하고 마지막에 핵심 키워드 3개를 붙여라:\n{text}",
    input_variables=["text"]
)
chain = LLMChain(llm=llm, prompt=prompt)
yt_summary = chain.run(transcript)

print(yt_summary)


## 3. 웹 검색 결합 RAG (Web Search-Augmented QA)


| 항목 | 설명 |
|------|------|
| 목적 | 실시간 검색을 통해 최신 정보를 LLM 응답에 반영 |
| 도구 | `TavilySearchResults` (TAVILY_API_KEY 필요) |
| 응용 | 뉴스 요약, 최신 논문/트렌드 요약 |


In [ ]:

# if not os.getenv("TAVILY_API_KEY"):
#     print(" TAVILY_API_KEY 환경변수가 필요합니다.")
# else:
#     search = TavilySearchResults(max_results=3)
#     llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

#     query = "2025년 인공지능 산업 전망에 대한 최신 소식"
#     results = search.run(query)
#     results_text = str(results)

#     prompt = PromptTemplate(
#         template=(
#             "다음 검색 결과를 바탕으로 핵심 동향을 6문장으로 요약하고, "
#             "출처를 항목으로 나열하라. 마지막에 한 줄로 한글 TL;DR를 추가하라.\n\n"
#             "[검색 결과]\n{context}\n"
#         ),
#         input_variables=["context"]
#     )
#     chain = LLMChain(llm=llm, prompt=prompt)
#     web_summary = chain.run(results_text)
#     print(web_summary)

## 4. 뉴스 / 블로그 자동 요약 (RSS)


| 항목 | 설명 |
|------|------|
| 목적 | RSS/뉴스 API로 기사 수집 후 자동 요약 |
| 라이브러리 | `feedparser`, `LangChain` |
| 예시 | 기술 뉴스 브리핑 자동 생성 |


In [ ]:

import feedparser

RSS_URL = "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml"
feed = feedparser.parse(RSS_URL)
type(feed)

In [ ]:
feed.entries

In [ ]:
items = []
for e in feed.entries[:3]:
    title = e.get("title", "")
    summary = e.get("summary", "")
    link = e.get("link", "")
    items.append(f"[{title}] {summary} (출처: {link})")

rss_text = "\n\n".join(items)
rss_text

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
prompt = PromptTemplate(
    template=(
        "다음 뉴스 3건을 항목별로 3문장 내외로 요약하고, 각각 한글 해시태그 2개를 붙여라.\n\n{news}"
    ),
    input_variables=["news"]
)
chain = LLMChain(llm=llm, prompt=prompt)
news_summary = chain.run(rss_text)
print(news_summary)

In [ ]:
from IPython.display import Markdown, display
Markdown(news_summary)

## 5. 멀티소스 통합 리포트 생성


YouTube 요약, 웹 검색 요약, RSS 뉴스 요약을 하나의 컨텍스트로 통합하여
**'AI 산업 트렌드 요약 리포트'**를 생성합니다.


In [ ]:

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

context = (
    "### [YouTube 요약]\n" + (yt_summary if 'yt_summary' in globals() else "") + "\n\n" +
    # "### [웹 검색 요약]\n" + (web_summary if 'web_summary' in globals() else "검색 요약 없음") + "\n\n" +
    "### [뉴스 요약]\n" + (news_summary if 'news_summary' in globals() else "")
)

prompt = PromptTemplate(
    template=(
        "다음 통합 컨텍스트를 바탕으로 'AI 산업 트렌드 요약 리포트'를 작성하라.\n"
        "- 5개의 핵심 트렌드(굵은 글씨로 제목)와 각 3~4문장 설명\n"
        "- 마지막에 '실무 적용 팁' 섹션을 불릿으로 5개 작성\n\n"
        "{context}"
    ),
    input_variables=["context"]
)
chain = LLMChain(llm=llm, prompt=prompt)
final_report = chain.run(context)
print(final_report)

In [ ]:
Markdown(final_report)

## 6. 실습 과제


1) 관심 있는 주제의 유튜브 영상을 선택해 요약을 생성한다.  
2) Tavily 검색을 통해 최신 기사/블로그를 3~5건 수집해 요약한다.  
3) 다른 RSS 피드(예: 경제/과학)를 바꿔 요약 파이프라인을 실행한다.  
4) 세 소스를 통합하여 주간 리포트를 자동 생성한다.  
5) (선택) 결과를 Markdown 파일로 저장하고, 날짜별 폴더 구조를 만든다.


## 참고 자료


- LangChain: https://python.langchain.com/docs/  
- Tavily tool: https://python.langchain.com/docs/integrations/tools/tavily  
- YouTube Transcript API: https://pypi.org/project/youtube-transcript-api/  
- NYTimes RSS: https://developer.nytimes.com/docs/rss-api/1/overview
